In [4]:
import pandas as pd

In [5]:
# data_dtm = pd.read_pickle('dtm.pkl')
# data_dtm = data_dtm.transpose()

In [6]:
df_billText = pd.read_csv('billText_Processed_n.csv',index_col=0)

In [7]:
df_billText.head()

,bill_id,long_title,bill_status
0,17SBN-2235,fiscal regime mining industry,Pending
1,17SBN-2234,sale certain land barangay city university cit...,Passed
2,17SBN-2233,excise tax tobacco tax incremental tobacco exc...,Passed
3,17SBN-2232,institutionalization development training orga...,Pending
4,17SBN-2231,bank,Pending


In [8]:
wordGroups = df_billText.long_title.tolist()
bill_words = [wordGroup.split(' ') for wordGroup in wordGroups]
flat_list = [item for sublist in bill_words for item in sublist]

In [9]:
flat_list

['fiscal',
 'regime',
 'mining',
 'industry',
 'sale',
 'certain',
 'land',
 'barangay',
 'city',
 'university',
 'city',
 'university',
 'charter',
 'excise',
 'tax',
 'tobacco',
 'tax',
 'incremental',
 'tobacco',
 'excise',
 'tax',
 'human',
 'resource',
 'development',
 'health',
 'internal',
 'revenue',
 'institutionalization',
 'development',
 'training',
 'organization',
 'administration',
 'basic',
 'reserve',
 'corp',
 'public',
 'private',
 'educational',
 'citizen',
 'force',
 'reservist',
 'service',
 'bank',
 'day',
 'remembrance',
 'road',
 'crash',
 'third',
 'day',
 'remembrance',
 'road',
 'crash',
 'nature',
 'park',
 'barangay',
 'city',
 'province',
 'responsible',
 'zone',
 'foreign',
 'disclosure',
 'pertinent',
 'secrecy',
 'evaluation',
 'protecting',
 'child',
 'marriage',
 'illegal',
 'energy',
 'consumer',
 'advocate',
 'sport',
 'academy',
 'training',
 'center',
 'barangay',
 'province',
 'toll',
 'certain',
 'participation',
 'civil',
 'society',
 'prepara

In [10]:
from collections import Counter

In [11]:
word_counter = Counter(flat_list)
most_common_words = word_counter.most_common()[:10]
most_common_words

[('public', 1243),
 ('development', 997),
 ('health', 919),
 ('education', 899),
 ('local', 852),
 ('system', 749),
 ('certain', 676),
 ('school', 611),
 ('service', 558),
 ('special', 530)]

In [12]:
from gensim import matutils, models
import scipy.sparse

In [13]:
# data_dtm.head()

In [14]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
# sparse_counts = scipy.sparse.csr_matrix(data_dtm)
# corpus = matutils.Sparse2Corpus(sparse_counts)

In [15]:
import pickle

In [16]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
# tfidfv = pickle.load(open("tfidf.pkl", "rb"))
# id2word = dict((v, k) for k, v in tfidfv.vocabulary_.items())

In [17]:
# id2word

In [18]:
df_lemmatized = pd.read_pickle('df_lemma.pkl')

In [19]:
df_lemmatized.head()

,bill_id,long_title,bill_status
0,17SBN-2235,"[fiscal, regime, mining, industry]",Pending
1,17SBN-2234,"[sale, certain, land, barangay, city, universi...",Passed
2,17SBN-2233,"[excise, tax, tobacco, tax, incremental, tobac...",Passed
3,17SBN-2232,"[institutionalization, development, training, ...",Pending
4,17SBN-2231,[bank],Pending


In [20]:
from gensim import corpora
# Create Dictionary
id2word = corpora.Dictionary(df_lemmatized.long_title)

# Create Corpus
texts = df_lemmatized.long_title

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [30]:
# LDA for num_topics = 15
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=15, passes=50, random_state=43)
lda.print_topics()

[(0,
  '0.071*"system" + 0.049*"assistance" + 0.048*"insurance" + 0.040*"establishment" + 0.036*"corporation" + 0.028*"credit" + 0.025*"emergency" + 0.024*"youth" + 0.022*"service" + 0.021*"computer"'),
 (1,
  '0.064*"center" + 0.054*"management" + 0.039*"hospital" + 0.037*"property" + 0.034*"compensation" + 0.030*"modernization" + 0.027*"promotion" + 0.026*"city" + 0.026*"provincial" + 0.021*"road"'),
 (2,
  '0.050*"province" + 0.041*"area" + 0.035*"book" + 0.032*"management" + 0.031*"mechanism" + 0.023*"real" + 0.023*"chapter" + 0.020*"academy" + 0.020*"estate" + 0.019*"regular"'),
 (3,
  '0.062*"family" + 0.047*"research" + 0.041*"public" + 0.037*"information" + 0.037*"right" + 0.035*"health" + 0.033*"prevention" + 0.021*"administration" + 0.020*"work" + 0.019*"control"'),
 (4,
  '0.112*"commission" + 0.054*"land" + 0.041*"agricultural" + 0.032*"development" + 0.029*"small" + 0.028*"medium" + 0.026*"transportation" + 0.021*"criminal" + 0.019*"consumer" + 0.018*"poor"'),
 (5,
  '0.06

In [31]:
from gensim.models import CoherenceModel

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=df_lemmatized.long_title, dictionary=id2word, coherence='c_v')


Perplexity:  -8.073226386172918


In [33]:
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4556193019044334


In [38]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

plt.style.use('seaborn-dark')


In [39]:
# Visualize the topics
pyLDAvis.enable_notebook()

In [40]:
plt.style.use('seaborn-dark')

vis = pyLDAvis.gensim.prepare(lda, corpus, id2word, R=10)
vis

/Users/emilolbinado/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
8      0.023868  0.158944       1        1  7.811055
14     0.185055 -0.062588       2        1  7.534307
11     0.140280 -0.226019       3        1  7.475564
7     -0.239541 -0.184107       4        1  7.380779
6      0.042625 -0.076358       5        1  7.108952
13    -0.107663  0.054467       6        1  7.045074
9     -0.058666  0.112248       7        1  6.813934
0      0.089102 -0.004276       8        1  6.749566
3      0.196543 -0.086118       9        1  6.639081
5      0.137409 -0.030883      10        1  6.585213
10     0.001839  0.177542      11        1  6.348168
4     -0.022228  0.172674      12        1  5.821261
12    -0.006639  0.186397      13        1  5.813523
1     -0.171426 -0.077020      14        1  5.486017
2     -0.210558 -0.114904      15        1  5.387506, topic_info=    Category         Freq        Term        Total  loglift  logprob
291  Default   950.000000   education   950.000000  10.0000  10.0000
121  Default   837.000000       local   837.000000   9.0000   9.0000
30   Default  1231.000000      public  1231.000000   8.0000   8.0000
13   Default   962.000000      health   962.000000   7.0000   7.0000
5    Default   738.000000     certain   738.000000   6.0000   6.0000
..       ...          ...         ...          ...      ...      ...
313  Topic15    59.887756    maritime    60.887810   2.9045  -4.2559
617  Topic15    98.866951        real   128.729660   2.6571  -3.7546
43   Topic15   212.324844    province   464.518433   2.1382  -2.9903
161  Topic15    96.996445     chapter   137.477936   2.5723  -3.7737
78   Topic15   133.866791  management   446.406769   1.7167  -3.4516

[247 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
684       2  0.990407       abuse
1283      3  0.967061    academic
58       15  0.990196     academy
186       4  0.899587  additional
186       5  0.062041  additional
...     ...       ...         ...
152       2  0.996175     welfare
520       9  0.715059        work
520      10  0.279213        work
159       2  0.058202       youth
159       8  0.931238       youth

[349 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 15, 12, 8, 7, 14, 10, 1, 4, 6, 11, 5, 13, 2, 3])

In [41]:
pyLDAvis.save_html(vis, 'lda_75_lem_5_pass_02.html')